In [19]:
import import_ipynb
import torch
import pandas as pd
import numpy as np
from config1 import model_config as config

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

import itertools
import matplotlib.pyplot as plt

In [20]:
print(config)

{'gpu': 0, 'bidirectional': False, 'input_dim': 9, 'hidden_dim': 50, 'output_dim': 6, 'dropout': 0.2, 'learning_rate': 0.01, 'batch_size': 1567, 'n_epochs': 55000, 'n_layers': 2, 'model_code': 'basic_lstm'}


In [13]:
def load_data(batched=True, test=False, file_dir='/home/rashmi/BE Project/DL/data/s2e/'):
    bs = config['batch_size']
    ftype = 'test' if test else 'train'
    df = pd.read_csv('{}modified_df_{}.csv'.format(file_dir, ftype))
    # 0th index in label, rest all are features
    data = (np.array(df[df.columns[3:]]), np.array(df[df.columns[0]]))
    if test or not batched:
        return [torch.FloatTensor(data[0]), torch.LongTensor(data[1])]
    data = list(zip(data[0], data[1]))
    n_iters = len(data) // bs
    batches = []
    for i in range(1, n_iters + 1):
        input_batch = []
        output_batch = []
        for e in data[bs * (i-1):bs * i]:
            input_batch.append(e[0])
            output_batch.append(e[1])
        batches.append([torch.FloatTensor(input_batch),
                        torch.LongTensor(output_batch)])
    return batches

In [14]:
def evaluate(targets, predictions):
    performance = {
        'acc': accuracy_score(targets, predictions),
        'f1': f1_score(targets, predictions, average='macro'),
        'precision': precision_score(targets, predictions, average='macro'),
        'recall': recall_score(targets, predictions, average='macro')}
    return performance

In [15]:
def plot_confusion_matrix(targets, predictions, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    #plt.figure(figsize=(8,8))
    cm = confusion_matrix(targets, predictions)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [16]:
import sys
import pickle
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from utils import load_data, evaluate, plot_confusion_matrix

In [17]:
class LSTMClassifier(nn.Module):
    """docstring for LSTMClassifier"""
    def __init__(self, config):
        super(LSTMClassifier, self).__init__()
        self.n_layers = config['n_layers']
        self.input_dim = config['input_dim']
        self.hidden_dim = config['hidden_dim']
        self.output_dim = config['output_dim']
        self.bidirectional = config['bidirectional']
        self.dropout = config['dropout'] if self.n_layers > 1 else 0

        self.rnn = nn.LSTM(self.input_dim, self.hidden_dim, bias=True,
                           num_layers=2, dropout=self.dropout,
                           bidirectional=self.bidirectional)
        self.out = nn.Linear(self.hidden_dim, self.output_dim)
        self.softmax = F.softmax

    def forward(self, input_seq):
        # input_seq =. [1, batch_size, input_size]
        rnn_output, (hidden, _) = self.rnn(input_seq)
        if self.bidirectional:  # sum outputs from the two directions
            rnn_output = rnn_output[:, :, :self.hidden_dim] +\
                        rnn_output[:, :, self.hidden_dim:]
        class_scores = F.softmax(self.out(rnn_output[0]), dim=1)
        return class_scores


In [18]:
if __name__ == '__main__':
    emotion_dict = {'ang': 0, 'hap': 1, 'sad': 2, 'fea': 3, 'sur': 4, 'neu': 5}

    device = 'cuda:{}'.format(config['gpu']) if \
             torch.cuda.is_available() else 'cpu'

    model = LSTMClassifier(config)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

    train_batches = load_data()
    test_pairs = load_data(test=True)

    best_acc = 0
    for epoch in range(config['n_epochs']):
        losses = []
        for batch in train_batches:
            inputs = batch[0].unsqueeze(0)  # frame in format as expected by model
            targets = batch[1]
            inputs = inputs.to(device)
            targets = targets.to(device)

            model.zero_grad()
            optimizer.zero_grad()

            predictions = model(inputs)
            predictions = predictions.to(device)

            loss = criterion(predictions, targets)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        # evaluate
        with torch.no_grad():
            inputs = test_pairs[0].unsqueeze(0)
            targets = test_pairs[1]

            inputs = inputs.to(device)
            targets = targets.to(device)

            predictions = torch.argmax(model(inputs), dim=1)  # take argmax to get class id
            predictions = predictions.to(device)

            # evaluate on cpu
            targets = np.array(targets.cpu())
            predictions = np.array(predictions.cpu())

            # Get results
            # plot_confusion_matrix(targets, predictions,
            #                       classes=emotion_dict.keys())
            performance = evaluate(targets, predictions)
            if performance['acc'] > best_acc:
                best_acc = performance['acc']
                print(performance)
                # save model and results
                torch.save({
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict()
                    }, 'runs/{}-best_model.pth'.format(config['model_code']))

                with open('results/{}-best_performance.pkl'.format(config['model_code']), 'wb') as f:
                    pickle.dump(performance, f)


      Unnamed: 0                wav_file  label  sig_mean   sig_std  \
0            195     Ses05F_impro03_M043      1  0.022838  0.036141   
1           1273  Ses05M_script02_1_F014      2  0.006699  0.011828   
2            770     Ses05M_impro08_F009      5  0.004782  0.007437   
3           1324  Ses05F_script03_2_F010      5  0.004031  0.006441   
4           2062     Ses05M_impro02_F017      3  0.005514  0.010058   
...          ...                     ...    ...       ...       ...   
1699         140     Ses05F_impro03_F049      1  0.005586  0.009389   
1700        2061     Ses05M_impro02_F016      3  0.003329  0.005647   
1701        1175  Ses05F_script01_1_M013      0  0.011628  0.017732   
1702        1458     Ses05F_impro02_F026      2  0.004980  0.009138   
1703        1581     Ses05M_impro04_M029      2  0.005442  0.012925   

      rmse_mean  rmse_std   silence  harmonic  auto_corr_max  auto_corr_std  
0      0.028836  0.021787  0.288089 -0.003263       0.916770       2.

IndexError: Target 195 is out of bounds.